In [15]:
import os
import requests
import json 
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
from pandas.tseries.offsets import DateOffset
from pandas import json_normalize
import time
from defillama import DefiLlama
from datetime import datetime
llama = DefiLlama()
import datetime as dt
import pytz


ETH, SOL, Base, SEI, SUI, INJ, Avax, Optimism

In [19]:
def get_net_flows(chain_name:str):
    ids = []
    aggregated_data = {}

    bridges = "https://bridges.llama.fi/bridges?includeChains=true"
    bridges = requests.get(bridges).json()
    
    for i in range(len(bridges['bridges'])):
        if f"{chain_name}" in bridges['bridges'][i]['chains']:
            ids.append(bridges['bridges'][i]['id'])
    
    for id in ids:
        url = f"https://bridges.llama.fi/bridgevolume/{chain_name}?id={id}"
        vol_data = requests.get(url).json()[-60:]

        for entry in vol_data:
            date = datetime.utcfromtimestamp(int(entry['date']))
            eastern = pytz.timezone("US/Eastern")
            date = date.replace(tzinfo=pytz.utc).astimezone(eastern)
            date = date.strftime('%Y-%m-%d')
            net_inflow = entry['depositUSD'] - entry['withdrawUSD']

            if date in aggregated_data:
                aggregated_data[date] += net_inflow
            else:
                aggregated_data[date] = net_inflow
    sorted_flows = sorted(aggregated_data.items(), key=lambda x: x[0])
    return sorted_flows

In [49]:
def get_stables(chain_name:str):
    stable_ids = []
    aggregated_stables = {}

    stables_url = "https://stablecoins.llama.fi/stablecoins?includePrices=true"
    stable_id = requests.get(stables_url).json()

    for i in range(len(stable_id['peggedAssets'])):
        stable_ids.append(stable_id['peggedAssets'][i]['id'])
    
    
    for id in stable_ids:
        url = F"https://stablecoins.llama.fi/stablecoincharts/{chain_name}?stablecoin={id}"
       
        stables_data = requests.get(url).json()[-60:]
        if len(stables_data) > 0:
       
            for entry in stables_data:
                date = datetime.utcfromtimestamp(int(entry['date']))
                eastern = pytz.timezone("US/Eastern")
                date = date.replace(tzinfo=pytz.utc).astimezone(eastern)
                date = date.strftime('%Y-%m-%d')

                if date in aggregated_stables:
                    aggregated_stables[date] += entry['totalCirculating']['peggedUSD']
                else:
                    aggregated_stables[date] = entry['totalCirculating']['peggedUSD']
        else:
            pass
    sorted_agg_stables = sorted(aggregated_stables.items(), key = lambda x:x[0])
    return sorted_agg_stables
    

In [52]:
def get_tvl(chain_name:str):
    tvl = {}
    tvl_url = F"https://api.llama.fi/v2/historicalChainTvl/{chain_name}"
    tvl_data = requests.get(tvl_url).json()[-60:]

    for entry in tvl_data:
        date = datetime.utcfromtimestamp(int(entry['date']))
        eastern = pytz.timezone("US/Eastern")
        date = date.replace(tzinfo=pytz.utc).astimezone(eastern)
        date = date.strftime('%Y-%m-%d')

        
        tvl[date] = entry['tvl']
    
    sorted_tvl = sorted(tvl.items(), key = lambda x:x[0])

    return sorted_tvl 

In [ ]:
def get_dex_vol():
    

In [20]:
# Works perfect
flows = get_net_flows("Base")
flows

[('2023-08-23', -14200),
 ('2023-08-24', -18300),
 ('2023-09-02', 22493),
 ('2023-09-03', 20391),
 ('2023-09-04', 18875),
 ('2023-09-05', -139561),
 ('2023-09-06', 35168),
 ('2023-09-07', 14966),
 ('2023-09-08', 2163),
 ('2023-09-09', 0),
 ('2023-09-10', -44252),
 ('2023-09-11', 0),
 ('2023-12-12', -1315089),
 ('2023-12-13', -880214),
 ('2023-12-14', 678564),
 ('2023-12-15', -1223),
 ('2023-12-16', -389645),
 ('2023-12-17', 1441277),
 ('2023-12-18', -16527),
 ('2023-12-19', 420747),
 ('2023-12-20', -123325),
 ('2023-12-21', 360014),
 ('2023-12-22', -20725),
 ('2023-12-23', -347989),
 ('2023-12-24', -1228460),
 ('2023-12-25', 1461363),
 ('2023-12-26', 2695495),
 ('2023-12-27', 783229),
 ('2023-12-28', -10166),
 ('2023-12-29', -2061385),
 ('2023-12-30', 1173834),
 ('2023-12-31', 1308024),
 ('2024-01-01', 1474411),
 ('2024-01-02', 1813352),
 ('2024-01-03', 3830154),
 ('2024-01-04', 1374970),
 ('2024-01-05', 1244989),
 ('2024-01-06', 803171),
 ('2024-01-07', 2613156),
 ('2024-01-08', 16800

In [50]:
# works perfect
stables = get_stables("Base")
stables 

In [53]:
# Works perfect
tvl = get_tvl('Base')
tvl

[('2023-12-12', 370266317.6125022),
 ('2023-12-13', 387152661.7283448),
 ('2023-12-14', 397666121.8108578),
 ('2023-12-15', 399264419.43908405),
 ('2023-12-16', 405678835.8462741),
 ('2023-12-17', 402537287.8680279),
 ('2023-12-18', 419396423.2817039),
 ('2023-12-19', 419343714.013779),
 ('2023-12-20', 443597658.20717096),
 ('2023-12-21', 450589267.63081485),
 ('2023-12-22', 448162223.74464345),
 ('2023-12-23', 451896696.9431839),
 ('2023-12-24', 452369902.736831),
 ('2023-12-25', 451713120.5597483),
 ('2023-12-26', 453029852.83770305),
 ('2023-12-27', 454395325.1113004),
 ('2023-12-28', 451576125.90438926),
 ('2023-12-29', 440119845.03400195),
 ('2023-12-30', 441349827.08224756),
 ('2023-12-31', 437157255.99282956),
 ('2024-01-01', 444171067.9471625),
 ('2024-01-02', 438070813.3351467),
 ('2024-01-03', 420751925.6382915),
 ('2024-01-04', 421371948.62472284),
 ('2024-01-05', 418966482.70274866),
 ('2024-01-06', 418422993.3203411),
 ('2024-01-07', 412480553.28625035),
 ('2024-01-08', 41